# CAPSTONE: Data Extraction from ERA5 .grib Files

**Author: Ishan Singh Bhullar**<br>
**Date: 11 December 2022**<br>
**Contact: ishanbhullar@gmail.com**<br>

*Note: Notebook run on Google Colaboratory due to memory issues on local machine.*

### Introduction

The dataset containing gridded geopotential (z) and wind (u,v) data was obtained from the Climate Data Store ([https://cds.climate.copernicus.eu/#!/home](https://cds.climate.copernicus.eu/#!/home)) via running a python file to interface with the API. Please refer to README_data.txt for more information. 

The data ranges from the year 1959 to 1999. Each individual .grib file corresponds to data for everyday of a given year between June and November for the North Atlantic Ocean and parts of North America. The data is bound between latitude (1 to 65) and logitude (2 to -133). The temporal range is from 0000hrs to 2100hrs with an interval of 3 hours. All the variables have readings available for 3 pressure levels 250hPa, 550hPa and 850hPa. The file for each year is  ~3.5 GB in size. The total data is ~140 GB. 

Given the sheer size of the data and the somewhat obscure format, I had to work with the **Xarray** library ([documentation](https://docs.xarray.dev/en/stable/)) and **cfgrib** python interface ([documentation](https://pypi.org/project/cfgrib/)).

At a high level, xarray uses cfgrib as an engine to read the .grib file into a *xarray dataset*. From there, I had to run a loop through the tracks dataset to extract information from the grib file based on time , latitude and longitude. Due to the size of the .grib files and cfrgib's propensity for memory leakage, I had to use Google Colaboratories to extract the relevant information. This meant first uploading all the .grib files to Google Drive.  

### Colab Commands to Access Files

In [ ]:
# to upload files from local system
from google.colab import files

In [ ]:
# to access and use files on google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Installing XARRAY and CFGRIB (with required dependencies)

In [ ]:
!pip install ecmwflibs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 74.4 MB 1.2 MB/s 
  Created wheel for findlibs: filename=findlibs-0.0.2-py3-none-any.whl size=6559 sha256=38f2310852e83c46d44482f5a6c8a704c9f8a84d659fbea1612411659d74e581
  Stored in directory: /root/.cache/pip/wheels/16/e7/71/9deaff72c4a92f111cbc38631e0829b26398ee3599aec65b1b
Successfully built findlibs


In [ ]:
!pip install eccodes==1.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.4 MB/s 
  Created wheel for eccodes: filename=eccodes-1.3.1-py3-none-any.whl size=39052 sha256=5460d4aa043c3a7e198219d57231b610e96b4845f5bf288d42464fe67134db24
  Stored in directory: /root/.cache/pip/wheels/58/45/ca/4d59fd6803aca50abdd251a9eb2eead51c9bcf24a796905b13
Successfully built eccodes


In [ ]:
!pip install importlib-metadata==4.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.13.0
    Uninstalling importlib-metadata-4.13.0:
      Successfully uninstalled importlib-metadata-4.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.4.1 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 4.0.1 which is incompatible.
gym 0.25.2 requires importlib-metadata>=4.8.0; python_version < "3.10", but you have importlib-metadata 4.0.1 which is incompatible.


In [ ]:
!pip install xarray==0.18.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 807 kB 5.0 MB/s 
  Attempting uninstall: xarray
    Found existing installation: xarray 0.20.2
    Uninstalling xarray-0.20.2:
      Successfully uninstalled xarray-0.20.2


In [ ]:
!pip install cfgrib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for cfgrib: filename=cfgrib-0.9.10.3-py3-none-any.whl size=46719 sha256=6abc102b291950f6275a6de31f5cdcbb35c62975c02f9dfe4c0938c7dd19ae24
  Stored in directory: /root/.cache/pip/wheels/27/28/8d/8b2e992d2d03933d9b1a04f20b7c157d22de6efabf3df4e48e
Successfully built cfgrib


### Importing Relevant Libraries

In [ ]:
import xarray as xr
import cfgrib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime 

In [ ]:
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings("ignore")

-------------------

### Main Code

Let's start with reading the cleaned tracks data.

In [ ]:
tracks_df = pd.read_csv("drive/My Drive/Capdata/capstone_track_data.csv", index_col=0)

# converting to standard datetime format
tracks_df['iso_time'] = pd.to_datetime(tracks_df['iso_time'])

The grib data we have pulled is only valid for months 6 to 11 as that is the peak season for Tropical cyclones. So I will clip the main tracks dataframe to only include data for months 6-11. Further, in interest of having equal time intervals within each storm, we will keep readings with a hour value of 0000hrs to 2100hrs with an interval of 3 hours. 

In [ ]:
hours_list = [0, 3, 6, 9, 12, 15, 18, 21] # list of valid hour values

# filtering data based on hours and months
tracks_df = tracks_df[(tracks_df['iso_time'].dt.month >= 6) & (tracks_df['iso_time'].dt.month <= 11)]
tracks_df = tracks_df[(tracks_df['iso_time'].dt.hour.isin(hours_list)) & (tracks_df['iso_time'].dt.minute == 0)]
tracks_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48642 entries, 31 to 50592
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   sid          48642 non-null  object        
 1   season       48642 non-null  int64         
 2   number       48642 non-null  int64         
 3   name         48642 non-null  object        
 4   iso_time     48642 non-null  datetime64[ns]
 5   lat          48642 non-null  float64       
 6   lon          48642 non-null  float64       
 7   dist2land    48642 non-null  int64         
 8   usa_status   48642 non-null  object        
 9   usa_wind     48642 non-null  float64       
 10  usa_pres     48642 non-null  float64       
 11  usa_sshs     48642 non-null  int64         
 12  storm_speed  48642 non-null  int64         
 13  storm_dir    48642 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(6), object(3)
memory usage: 5.6+ MB


In [ ]:
start_year = 1959 # starting year of .grib files
end_year = 1999 # starting year of .grib files
pres_levels = [250, 550, 850] # three pressure levels for the .grib data

for year in range(start_year, end_year + 1):
  print(f'Loop for Year: {year}')
  grib_data = xr.open_dataset("drive/My Drive/Capdata/era5_"+str(year)+".grib", engine='cfgrib', backend_kwargs={'indexpath': ''})

  # clip tracks_df based on the year value from `iso_time` column
  clip_df = tracks_df[(tracks_df['iso_time'].dt.year == year)]

  # loop for pressure levels
  for level in pres_levels:

      # create empty lists to store variables from grib_data
      geo_z = [] # geopotential
      wind_u = [] # horizontal component of wind
      wind_v = [] # vertical component of wind

      # loop to cycle through the clips_df dataframe row by row and extract values from grib_data based on `time`, `lat` and `lon`
      for row in tqdm(clip_df.index):
  
          time = clip_df['iso_time'][row]
          lat = round(clip_df['lat'][row])
          lon = round(clip_df['lon'][row])

          # extracting data from grib_data, converting it to a numpy array and flattening it to get scalar value
          z = np.asarray(np.mean(grib_data.loc[dict(time=str(time),
                                                    isobaricInhPa=level, latitude=range(lat-2, lat+3), longitude=range(lon-2, lon+3))]['z'])).flat[0]
          geo_z.append(z)
          u = np.asarray(np.mean(grib_data.loc[dict(time=str(time),
                                                    isobaricInhPa=level, latitude=range(lat-2, lat+3),
                                                    longitude=range(lon-2, lon+3))]['u'])).flat[0]
          wind_u.append(u)
          v = np.asarray(np.mean(grib_data.loc[dict(time=str(time),
                                                    isobaricInhPa=level, latitude=range(lat-2, lat+3),
                                                    longitude=range(lon-2, lon+3))]['v'])).flat[0]
          wind_v.append(v)
      
      # adding extracted list of values from grib_data to clip_df as a new column
      clip_df['geo_'+str(level)] = geo_z
      clip_df['u_wind_'+str(level)] = wind_u
      clip_df['v_wind_'+str(level)] = wind_v

  clip_df.to_csv('drive/My Drive/Capdata/csvFiles/'+str(year)+'_data.csv')

Loop for Year: 1963


100%|██████████| 577/577 [00:24<00:00, 23.88it/s]


Loop for Year: 1964


100%|██████████| 797/797 [00:42<00:00, 18.55it/s]


Loop for Year: 1965


...

100%|██████████| 331/331 [00:13<00:00, 24.02it/s]


Loop for Year: 1998


100%|██████████| 867/867 [00:35<00:00, 24.11it/s]


Loop for Year: 1999


100%|██████████| 839/839 [00:41<00:00, 20.43it/s]
